In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3403/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_10_49_9,0.999872,0.645157,0.680404,0.999816,0.000041,0.210650,0.223883,0.000065,0.002205,0.006377,1.000025,0.006426,1606.220394,3880.175213,"Hidden Size=[36, 18], regularizer=0.02, learni..."
1,model_10_49_8,0.999871,0.645089,0.680434,0.999821,0.000041,0.210691,0.223863,0.000063,0.002212,0.006392,1.000025,0.006442,1606.210741,3880.165560,"Hidden Size=[36, 18], regularizer=0.02, learni..."
2,model_10_49_7,0.999871,0.645022,0.680453,0.999826,0.000041,0.210730,0.223849,0.000062,0.002220,0.006413,1.000025,0.006463,1606.197935,3880.152754,"Hidden Size=[36, 18], regularizer=0.02, learni..."
3,model_10_49_6,0.999870,0.644944,0.680415,0.999830,0.000041,0.210777,0.223876,0.000060,0.002227,0.006434,1.000025,0.006484,1606.184672,3880.139491,"Hidden Size=[36, 18], regularizer=0.02, learni..."
4,model_10_49_5,0.999869,0.644862,0.680387,0.999834,0.000042,0.210825,0.223896,0.000058,0.002235,0.006456,1.000025,0.006507,1606.170688,3880.125507,"Hidden Size=[36, 18], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,model_8_0_0,-0.087181,-0.135033,-0.166604,-0.211594,0.345326,0.673804,0.817230,0.282537,0.292114,0.587645,-16.530800,0.592218,244.126532,591.098201,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
4992,model_3_0_2,-0.464341,-0.173371,-0.304336,-0.596225,0.465125,0.696564,0.913714,0.521178,0.305259,0.682001,-1.361250,0.687308,99.530898,240.040086,"Hidden Size=[12], regularizer=0.02, learning_r..."
4993,model_3_0_1,-0.492856,-0.185561,-0.314499,-0.630716,0.474182,0.703800,0.920834,0.532440,0.307835,0.688609,-1.407231,0.693968,99.492327,240.001515,"Hidden Size=[12], regularizer=0.02, learning_r..."
4994,model_3_0_0,-0.515344,-0.197485,-0.327381,-0.656750,0.481325,0.710879,0.929858,0.540940,0.310652,0.693776,-1.443492,0.699175,99.462424,239.971612,"Hidden Size=[12], regularizer=0.02, learning_r..."
